# **ETL de arquivo CSV do portal de dados abertos MEC**: "Unidades da Rede Federal de EPCT"

### **Fluxo de trabalho:**

1. Extrair CSV oficial do portal do MEC (OData Olinda MEC).

2. Tratar textos (mojibake, acentuação, Unicode), padronizar colunas e enriquecer com Endereço e Regiões.

3. Carregar o DataFrame em uma aba do Google Sheets (link público).

In [1]:
# Bibliotecas necessárias
from pathlib import Path
import pandas as pd
import requests
import io
from io import BytesIO
import unicodedata
import re

# Autenticação no Google Sheets
from google.colab import auth
import gspread
import google.auth
from gspread import Client
print("Importação de bibliotecas concluída.")

Importação de bibliotecas concluída.


In [2]:
# @title **ETAPA 1: Download, tratamento e exibição dos dados da RFEPT**

"""
Leitura de CSV dados abertos MEC: "Unidades da Rede Federal de EPCT"
- Faz download do arquivo direto da URL oficial
- Detecta/força codificação adequada (UTF-8/UTF-8-SIG/CP1252/Latin-1)
- Corrige possíveis casos de "mojibake" (ex.: "Ã¡, Ã£, Ã©, Ãº, Ã¶")
- Auto-detecta separador (muitos CSVs do MEC usam ';')
- Exibe dimensões (n_linhas x n_colunas) e uma prévia do DataFrame

Uso sugerido (Jupyter/Colab):
    %run -i mec_unidades_rede_federal_csv_leitura_robusta.py

Ou importe a função principal e chame diretamente:
    from mec_unidades_rede_federal_csv_leitura_robusta import load_mec_unidades
    df = load_mec_unidades(show_preview=True)
"""

# ----------------------------
#  Download, tratamento e exibição dos dados dos IFs e Campus
# ----------------------------

# Download do Portal de Dados Abertos do MEC
# https://dados.gov.br/dados/conjuntos-dados/rede-federal-de-educacao-profissional-cientifica-e-tecnologica
URL = (
    "https://olinda.mec.gov.br/olinda-ide/servico/PDA_SETEC/versao/v1/odata/"  # Link específico
    "RedeFederal_Lista_Instituicoes_RFEPCT_2022?$format=text/csv"
)

# Cria o diretório de saída dentro do ambiente Colab
OUT_DIR = Path.cwd() / "dados_mec"
OUT_DIR.mkdir(exist_ok=True)
CSV_PATH = OUT_DIR / "Unidades_Rede_Federal_EPCT.csv"

# ----------------------------
# Utilidades de codificação e correção
# ----------------------------

def _has_utf8_bom(b: bytes) -> bool:
    """Verifica se os bytes possuem a Marca de Ordem de Byte (BOM) UTF-8."""
    return b.startswith(b"\xEF\xBB\xBF")


def _looks_mojibaked(text: str) -> bool:
    """Heurística simples para detectar mojibake típico (UTF-8 lido como Latin-1)."""
    # Ex.: 'Ã¡', 'Ã£', 'Ã©', 'Ã³', 'Ãº', 'Ã§', 'Ã±', 'Â', ''
    return bool(re.search(r"Ã.|Â|", text))


def _demojibake(s: str) -> str:
    """Tenta corrigir mojibake comum recodificando latin1 -> utf-8.
    Se falhar, retorna o original.
    """
    try:
        # A recodificação corrige os caracteres
        return s.encode("latin1", errors="ignore").decode("utf-8", errors="ignore")
    except Exception:
        return s


def _normalize_unicode(s: str) -> str:
    """Normaliza Unicode (útil para acentuação consistente)."""
    return unicodedata.normalize("NFC", s)


def _fix_dataframe_text(df: pd.DataFrame) -> pd.DataFrame:
    """Aplica correção de mojibake e normalização de Unicode em colunas de texto,
    incluindo a correção específica para 'ÿ'."""
    df = df.copy() # Evita SettingWithCopyWarning
    obj_cols = df.select_dtypes(include=["object"]).columns
    for col in obj_cols:
        # Garante que é string
        s = df[col].astype(str).fillna('')

        # Correção específica para 'ÿ' substituindo por 'ô' antes de verificar mojibake
        s = s.str.replace('ÿ', 'ô', regex=False)

        # Corrige mojibake se detectado
        mask = s.apply(_looks_mojibaked)
        if mask.any():
            s[mask] = s[mask].map(_demojibake)


        # Normaliza Unicode
        df[col] = s.map(_normalize_unicode)


    return df

# ----------------------------
# Download e leitura
# ----------------------------

def _download_bytes(url: str, timeout: int = 60) -> bytes:
    """Faz o download dos dados brutos."""
    with requests.get(url, stream=True, timeout=timeout) as r:
        r.raise_for_status()
        return r.content


def _try_read_csv_from_bytes(data: bytes, encoding: str) -> pd.DataFrame:
    """Tenta ler o CSV a partir dos bytes, auto-detectando o separador."""
    # sep=None + engine='python' para autodetectar ',' ou ';'
    # O arquivo do MEC usa ponto-e-vírgula ';', este método é mais robusto
    return pd.read_csv(BytesIO(data), sep=None, engine="python", encoding=encoding)


# ----------------------------
# Funções de tratamento
# ----------------------------

def remove_campus_suffixes(df: pd.Series) -> pd.Series:
    """
    Remova sufixos de institutos federais da coluna 'Campus' de forma otimizada.

    Utiliza uma única expressão regular para encontrar e remover os padrões
    apenas no final da string, garantindo eficiência e precisão.
    """
    # Cria um padrão regex que une todas as siglas dos IFs.
    # \s*-\s* -> busca pelo hífen com ou sem espaços ao redor.
    # (CPII|IF Baiano|IFMA|IFMG|IFPB|IFPR|IFRJ|IFSP|IFS|IFTO) -> grupo que captura qualquer uma das siglas listadas.
    # $ -> âncora que garante que a busca seja feita apenas no final da string.
    regex_pattern = r'\s*-\s*(CPII|IF Baiano|IFMA|IFMG|IFPB|IFPR|IFRJ|IFSP|IFS|IFTO)$'

    # Executa a substituição uma única vez para todo o DataFrame
    return df.str.replace(regex_pattern, "", regex=True).str.strip()


def create_address_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cria a coluna 'Endereço' mesclando Município e UF com '/'.
    Assume que as colunas originais 'NOME_MUNICIPIO_UNIDADE_ENSINO' e 'SIGLA_UF_UNIDADE_ENSINO' existem.
    """
    df = df.copy()
    municipio_col = 'NOME_MUNICIPIO_UNIDADE_ENSINO'
    uf_col = 'SIGLA_UF_UNIDADE_ENSINO'

    # Verifica se as colunas necessárias existem
    if municipio_col in df.columns and uf_col in df.columns:
        # Converte para string e preenche NaNs com string vazia antes de combinar
        df['Endereço'] = df[municipio_col].astype(str).fillna('') + '/' + df[uf_col].astype(str).fillna('')
    else:
        print(f"Aviso: Colunas '{municipio_col}' ou '{uf_col}' não encontradas para criar a coluna 'Endereço'.")

    return df


def add_regioes_column(df: pd.DataFrame) -> pd.DataFrame:
    """Adiciona coluna 'Regiões' com base na UF brasileira."""
    mapa_regioes = {
        # Norte
        'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte',
        'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
        # Nordeste
        'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste',
        'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste',
        'RN': 'Nordeste', 'SE': 'Nordeste',
        # Centro-Oeste
        'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
        # Sudeste
        'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
        # Sul
        'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
    }

    df = df.copy()
    df['Regiões'] = df['UF'].str.upper().map(mapa_regioes)
    return df


def load_mec_unidades(url: str = URL, save_local: bool = True, show_preview: bool = True) -> pd.DataFrame:
    """Baixa e lê o CSV do MEC com heurísticas de codificação e correção de mojibake,
    adiciona a coluna 'Regiões' e cria a coluna 'Endereço'.
    """

    try:
        raw = _download_bytes(url)
    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Falha no download da URL {url}: {e}")

    # Salva os bytes crus (sem transformação)
    if save_local:
        CSV_PATH.write_bytes(raw)

    # Ordem de tentativas de codificação
    candidates = ["utf-8-sig", "utf-8", "cp1252", "latin-1"]
    if _has_utf8_bom(raw):
        # Prioriza utf-8-sig se o BOM for detectado
        candidates = ["utf-8-sig", "utf-8", "cp1252", "latin-1"]

    last_err = None
    used_encoding = None
    df = None

    for enc in candidates:
        try:
            df = _try_read_csv_from_bytes(raw, enc)
            used_encoding = enc
            break
        except (UnicodeDecodeError, pd.errors.ParserError) as e:
            last_err = e
        except Exception as e:
            last_err = e
            continue

    if df is None:
        raise RuntimeError(f"Falha ao decodificar CSV ou determinar separador. Último erro: {last_err}")

    # Correções de texto
    df = _fix_dataframe_text(df)

    # Aplica a remoção de sufixos no Campus
    if 'NOME_UNIDADE_ENSINO' in df.columns:
        df['NOME_UNIDADE_ENSINO'] = remove_campus_suffixes(df['NOME_UNIDADE_ENSINO'])

    # Cria a coluna 'Endereço' ANTES de renomear as colunas originais usadas
    df = create_address_column(df)

    # 1. Mapeamento das colunas originais para os novos nomes
    rename_map = {
        'ANO': 'Ano_Criação',
        'SIGLA_INSTITUICAO': 'Sigla_IF',
        'NOME_INSTITUICAO': 'Nome_IF',
        'NOME_UNIDADE_ENSINO': 'Campus_IF',
        'NOME_MUNICIPIO_UNIDADE_ENSINO': 'Município',
        'SIGLA_UF_UNIDADE_ENSINO': 'UF',
    }

    # Renomeia as colunas
    df = df.rename(columns=rename_map)

    # Adiciona a coluna 'Regiões'
    df = add_regioes_column(df)

    # Ordena o DataFrame por 'UF' e 'Nome_IF'
    if 'UF' in df.columns and 'Nome_IF' in df.columns:
        df = df.sort_values(by=['UF', 'Nome_IF']).reset_index(drop=True)
    else:
        print("Aviso: Colunas 'UF' ou 'Nome_IF' não encontradas para ordenação.")


    # Lista as colunas a serem mantidas na ordem desejada, incluindo a nova coluna 'Endereço' e 'Regiões'
    colunas_desejadas_final = [
        'Ano_Criação',
        'Sigla_IF',
        'Nome_IF',
        'Campus_IF',
        'Regiões',
        'Município',
        'UF',
        'Endereço'
    ]

    # Seleciona e reordena apenas as colunas desejadas e renomeadas
    df = df[[col for col in colunas_desejadas_final if col in df.columns]]

    # Informação e prévia
    if show_preview:
        n_linhas, n_colunas = df.shape
        print("-" * 80)
        print("Dimensão da Rede Federal de Educação Profissional, Científica e Tecnológica (RFEPCT)")
        print("-" * 80)
        print(f"URL: {url}")
        print(f"Codificação utilizada: {used_encoding}")
        print(f"Dimensões: {n_linhas} linhas x {n_colunas} colunas\n")
        print("-" * 80)
        print("EXIBIR PRIMEIRAS LINHAS DE DATAFRAME `df_rede_federal`")
        display(df.head(15))
        print("-" * 80)
        print("TIPOS DE DADOS E VALORES NÃO-NULOS:")
        df.info()

    return df

if __name__ == "__main__":
    df_rede_federal = load_mec_unidades(show_preview=True)

--------------------------------------------------------------------------------
Dimensão da Rede Federal de Educação Profissional, Científica e Tecnológica (RFEPCT)
--------------------------------------------------------------------------------
URL: https://olinda.mec.gov.br/olinda-ide/servico/PDA_SETEC/versao/v1/odata/RedeFederal_Lista_Instituicoes_RFEPCT_2022?$format=text/csv
Codificação utilizada: utf-8-sig
Dimensões: 644 linhas x 8 colunas

--------------------------------------------------------------------------------
EXIBIR PRIMEIRAS LINHAS DE DATAFRAME `df_rede_federal`


,Ano_Criação,Sigla_IF,Nome_IF,Campus_IF,Regiões,Município,UF,Endereço
0,2010,IFAC,Instituto Federal do Acre,Campus Rio Branco,Norte,Rio Branco,AC,Rio Branco/AC
1,2010,IFAC,Instituto Federal do Acre,Campus Sena Madureira,Norte,Sena Madureira,AC,Sena Madureira/AC
2,2013,IFAC,Instituto Federal do Acre,Campus Tarauacá,Norte,Tarauacá,AC,Tarauacá/AC
3,2010,IFAC,Instituto Federal do Acre,Campus Cruzeiro do Sul,Norte,Cruzeiro do Sul,AC,Cruzeiro do Sul/AC
4,2013,IFAC,Instituto Federal do Acre,Campus Xapuri,Norte,Xapuri,AC,Xapuri/AC
5,2014,IFAC,Instituto Federal do Acre,Campus Rio Branco Baixada do Sol,Norte,Rio Branco,AC,Rio Branco/AC
6,2010,IFAL,Instituto Federal de Alagoas,Campus Penedo,Nordeste,Penedo,AL,Penedo/AL
7,2012,IFAL,Instituto Federal de Alagoas,Campus Piranhas,Nordeste,Piranhas,AL,Piranhas/AL
8,2013,IFAL,Instituto Federal de Alagoas,Campus Santana do Ipanema,Nordeste,Santana do Ipanema,AL,Santana do Ipanema/AL
9,2013,IFAL,Instituto Federal de Alagoas,Campus São Miguel dos Campos,Nordeste,São Miguel dos Campos,AL,São Miguel dos Campos/AL


--------------------------------------------------------------------------------
TIPOS DE DADOS E VALORES NÃO-NULOS:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ano_Criação  644 non-null    int64 
 1   Sigla_IF     644 non-null    object
 2   Nome_IF      644 non-null    object
 3   Campus_IF    644 non-null    object
 4   Regiões      644 non-null    object
 5   Município    644 non-null    object
 6   UF           644 non-null    object
 7   Endereço     644 non-null    object
dtypes: int64(1), object(7)
memory usage: 40.4+ KB


In [ ]:
# @title **ETAPA 2. Exportar DataFrame `df_rede_federal`para o Google Sheets**

# ======================== Configurações da Planilha ========================
# ID da planilha
ID_PLANILHA = '19JkJDxBOuuoED-IMt07ofEKWy5fxnIOf5BbZikM2HPM' # Link público
# Nome da aba. Se não existir, será criada.
NOME_ABA = 'Mapa_RFEPCT'
# ==============================================================

# 1. Autenticação do Google Colab
print("Autenticando-se no Google...")
# Inicia autenticação no Google
auth.authenticate_user()
print("Autenticação concluída.")

# 2. Inicializar o cliente gspread
# Utilizar as credenciais padrão estabelecidas pelo auth.authenticate_user()
credentials, project = google.auth.default()
gc = Client(auth=credentials)

try:
    # 3. Abrir a planilha usando o ID (Chave)
    sh = gc.open_by_key(ID_PLANILHA)
    print(f"Planilha ID: '{ID_PLANILHA}' acessada com sucesso.")

    # 4. Selecionar ou criar a aba (worksheet)
    try:
        # Tenta selecionar a aba existente
        worksheet = sh.worksheet(NOME_ABA)
        # Limpa o conteúdo existente para substituí-lo
        worksheet.clear()
        print(f"Aba '{NOME_ABA}' limpa e pronta para receber os novos dados.")
    except gspread.exceptions.WorksheetNotFound:
        # Se não encontrar, adiciona uma nova aba
        # Especificar número de linhas/colunas para garantir espaço
        worksheet = sh.add_worksheet(title=NOME_ABA, rows="1000", cols="10")
        print(f"Aba '{NOME_ABA}' criada.")


    # 5. Converter o DataFrame para uma lista de listas (formato que o gspread usa)
    # Inclui o cabeçalho (nomes das colunas)
    # Usar .tolist() (sem underscore) para converter o array NumPy
    # Certifica-se que df_rede_federal está definido e não vazio
    if 'df_rede_federal' in locals() and not df_rede_federal.empty:
        dados_para_sheet = [df_rede_federal.columns.tolist()] + df_rede_federal.values.tolist()
    else:
        raise ValueError("DataFrame 'df_rede_federal' não encontrado ou está vazio.")


    # 6. Exportar os dados (atualiza toda a faixa a partir de A1)
    # Usar argumentos nomeados para evitar o DeprecationWarning
    worksheet.update(values=dados_para_sheet, range_name='A1')

    # 7. Exibir o link
    print("-" * 50)
    print("Exportação concluída com sucesso!")
    print(f"Acesse a planilha aqui: {sh.url}")
    print("-" * 50)

except gspread.exceptions.SpreadsheetNotFound:
    print("-" * 50)
    print(f"ERRO: Planilha com ID '{ID_PLANILHA}' não encontrada.")
    print("Verifique se o ID está correto ou se você possui acesso a ela.")
    print("-" * 50)
except gspread.exceptions.APIError as e:
    print("-" * 50)
    print("ERRO DE API (Permissão):")
    print("Verifique se a conta Google usada na autenticação tem permissão de 'Editor' na planilha.")
    print(f"Detalhes do erro: {e}")
    print("-" * 50)
except Exception as e:
    # Captura outros erros, incluindo problemas de permissão
    print(f"Ocorreu um erro inesperado durante a exportação: {e}")

Autenticando-se no Google...
Autenticação concluída.
Planilha ID: '19JkJDxBOuuoED-IMt07ofEKWy5fxnIOf5BbZikM2HPM' acessada com sucesso.
Aba 'Mapa_RFEPCT' criada.
--------------------------------------------------
Exportação concluída com sucesso!
Acesse a planilha aqui: https://docs.google.com/spreadsheets/d/19JkJDxBOuuoED-IMt07ofEKWy5fxnIOf5BbZikM2HPM
--------------------------------------------------
